In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from google.colab import drive
drive.mount('drive')

spam_txt = pd.read_csv('/content/drive/MyDrive/COE 005/HW1/spam.csv', encoding="ISO-8859-1")
spam_txt


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
data = spam_txt.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
spam_data = data.rename(columns={"v1":'label', "v2":'Text'})
print(spam_data.head())
tags = spam_data["label"]
texts = spam_data["Text"]

  label                                               Text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


### To do: develop an accurate simple neural network model for spam classification (no LSTM, CNN, etc.)

In [ ]:
spam_data['label_enc'] = spam_data['label'].map({'ham':0,'spam':1})
spam_data.head()

,label,Text,label_enc
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
avg_words_len=round(sum([len(i.split()) for i in spam_data['Text']])/len(spam_data['Text']))
print(avg_words_len)

15


In [ ]:
s = set()
for sent in spam_data['Text']:
  for word in sent.split():
    s.add(word)
total_words_length=len(s)
print(total_words_length)

15585


In [ ]:
from sklearn.model_selection import train_test_split
  
X, y = np.asanyarray(spam_data['Text']), np.asanyarray(spam_data['label_enc'])
new_df = pd.DataFrame({'Text': X, 'label': y})
X_train, X_test, y_train, y_test = train_test_split(
    new_df['Text'], new_df['label'], test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4457,), (4457,), (1115,), (1115,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
  
tfidf_vec = TfidfVectorizer().fit(X_train)
X_train_vec,X_test_vec = tfidf_vec.transform(X_train),tfidf_vec.transform(X_test)
  
baseline_model = MultinomialNB()
baseline_model.fit(X_train_vec,y_train)

MultinomialNB()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
  
def compile_model(model):
    '''
    simply compile the model with adam optimzer
    '''
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
  
def fit_model(model, epochs, X_train=X_train, y_train=y_train,
              X_test=X_test, y_test=y_test):
    '''
    fit the model with given epochs, train 
    and test data
    '''
    history = model.fit(X_train,
                        y_train,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        validation_steps=int(0.2*len(X_test)))
    return history
  
def evaluate_model(model, X, y):
    '''
    evaluate the model and returns accuracy, 
    precision, recall and f1-score 
    '''
    y_preds = np.round(model.predict(X))
    accuracy = accuracy_score(y, y_preds)
    precision = precision_score(y, y_preds)
    recall = recall_score(y, y_preds)
    f1 = f1_score(y, y_preds)
  
    model_results_dict = {'accuracy': accuracy,
                          'precision': precision,
                          'recall': recall,
                          'f1-score': f1}
  
    return model_results_dict

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
  
# model with Sequential api
model_3 = keras.Sequential()
  
# universal-sentence-encoder layer 
# directly from tfhub
use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                           trainable=False,
                           input_shape=[],
                           dtype=tf.string,
                           name='USE')
model_3.add(use_layer)
model_3.add(layers.Dropout(0.2))
model_3.add(layers.Dense(64, activation=keras.activations.relu))
model_3.add(layers.Dense(1, activation=keras.activations.sigmoid))
  
compile_model(model_3)
  
history_3 = fit_model(model_3, epochs=5)

Epoch 1/5
140/140 [==============================] - 8s 33ms/step - loss: 0.2998 - accuracy: 0.9138 - val_loss: 0.1161 - val_accuracy: 0.9668
Epoch 2/5
140/140 [==============================] - 3s 20ms/step - loss: 0.0815 - accuracy: 0.9773 - val_loss: 0.0713 - val_accuracy: 0.9758
Epoch 3/5
140/140 [==============================] - 3s 20ms/step - loss: 0.0568 - accuracy: 0.9838 - val_loss: 0.0595 - val_accuracy: 0.9803
Epoch 4/5
140/140 [==============================] - 2s 17ms/step - loss: 0.0470 - accuracy: 0.9861 - val_loss: 0.0545 - val_accuracy: 0.9821
Epoch 5/5
140/140 [==============================] - 3s 20ms/step - loss: 0.0415 - accuracy: 0.9879 - val_loss: 0.0527 - val_accuracy: 0.9821


In [ ]:
new_df.to_csv('/content/drive/MyDrive/COE 005/HW1/danao_spam_output.csv')